In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#for interactivity
from ipywidgets import interact

In [ ]:
df= pd.read_csv('../input/crop-recommendation-dataset/Crop_recommendation.csv')
df.shape

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df['label'].value_counts()

In [ ]:
df.describe()

In [ ]:
@interact
def summary(crops = list(df['label'].value_counts().index)):
    x = df[df['label']== crops]
    z = df.drop(['label'],axis=1)
    y = []
    y.append(z)
    for i in z:
        print('Minimum', i, 'required', x[i].min())
        print('Average', i, 'required', x[i].mean())
        print('Maximum', i, 'required', x[i].max())
        print('--------------------------------------------------')

In [ ]:
df.skew()

In [ ]:
df.hist(figsize=(15,15), layout=(4,4), bins=20)

observation:
1. P is right skewed
2. Humidity left skewed

In [ ]:
print('crops which requires very high ratio of Nitrogen content in soil:', df[df['N']>120]['label'].unique())
print('crops which requires very high ratio of Phosphorous content in soil:', df[df['P']>100]['label'].unique())
print('crops which requires very high ratio of Potassium content in soil:', df[df['K']>200]['label'].unique())
print('crops which requires very high rainfall:', df[df['rainfall']>200]['label'].unique())
print('crops which requires very low temperature:', df[df['temperature']<10]['label'].unique())
print('crops which requires very high temperature:', df[df['temperature']>40]['label'].unique())
print('crops which requires very low humidity:', df[df['humidity']<20]['label'].unique())
print('crops which requires very low ph:', df[df['ph']<4]['label'].unique())
print('crops which requires very high ph:', df[df['ph']>9]['label'].unique())

observation:

we can see mothbeens grow in very low and very high pH conditions so pH is not a major factor for mothbeens

In [ ]:
print('Summer Crops:', df[(df['temperature']>30) & (df['humidity']>50)]['label'].unique())
print('Winter Crops:', df[(df['temperature']<20) & (df['humidity']>30)]['label'].unique())
print('Rainy Crops:', df[(df['rainfall']>200) & (df['humidity']>30)]['label'].unique())

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
z = df.drop(['label'],axis=1)
z = df.loc[:,z.columns].values
x_df = pd.DataFrame(z)
x_df.head()

In [ ]:
#Determine Optimum number of cluster by elbow method
from sklearn.cluster import KMeans
plt.rcParams['figure.figsize'] = (10,4)
wcss = []
for i in range (1,11):
    km = KMeans(n_clusters =i, init= 'k-means++', max_iter=300, n_init=10, random_state=0)
    km.fit(z)
    wcss.append(km.inertia_)
    
#plot the results
plt.plot(range(1,11), wcss)
plt.title('Elbow Method', fontsize= 15)
plt.xlabel('No. of cluster')
plt.ylabel('wcss')
plt.show()

Observation:

we get two elbows at 3 and 4. As per elbow method definition we take the last one
so our no. of cluster will be #4

In [ ]:
km = KMeans(n_clusters =4, init= 'k-means++', max_iter=300, n_init=10, random_state=0)
y_means= km.fit_predict(z)

a = df['label']
y_means = pd.DataFrame(y_means)
w = pd.concat([y_means, a], axis=1)
w =w.rename(columns= {0:'cluster'})

In [ ]:
#Check Cluster of each group

for i in range(0,4): #for 4 clusters 0,1,2,3
    print('Crops is cluster', i, w[w['cluster']==i]['label'].unique())
    print('---------------------------------------------------------------------------------------')

Observation:

some of the crops are more than in one cluster. This is known as soft clustering

We want our crops to be in single cluster so next step we are going to do hard clustering

In hard clustering we can able to see each data points are completely in one cluster or not

In [ ]:
for i in range(0,4):
    counts=w[w['cluster']==i]['label'].value_counts()
    d = w.loc[w['label'].isin(counts.index[counts>=50])]
    d=d['label'].value_counts()
    print('Crops in Cluster', i,':', list(d.index))
    print('--------------------------------------------------------------------------------')

In [ ]:
plt.figure(figsize=(15, 28))
plt.subplot(4,2,1)
sns.barplot(df['N'], df['label'])
plt.ylabel(' ')
plt.xlabel('Nitrogen')
plt.subplot(4,2,2)
sns.barplot(df['P'], df['label'])
plt.ylabel(' ')
plt.xlabel('Phophorus')
plt.subplot(4,2,3)
sns.barplot(df['K'], df['label'])
plt.ylabel(' ')
plt.xlabel('Potassium')
plt.subplot(4,2,4)
sns.barplot(df['temperature'], df['label'])
plt.ylabel(' ')
plt.xlabel('Temperature')
plt.subplot(4,2,5)
sns.barplot(df['humidity'], df['label'])
plt.ylabel(' ')
plt.xlabel('Humidity')
plt.subplot(4,2,6)
sns.barplot(df['ph'], df['label'])
plt.ylabel(' ')
plt.xlabel('pH')
plt.subplot(4,2,7)
sns.barplot(df['rainfall'], df['label'])
plt.ylabel(' ')
plt.xlabel('rainfall')
#apply for loop


**Observation**
1. Cotton requires high amount of Nitrogen among all
2. Grapes and Apple requires very high amount of phosphorus and Potassium
3. least amount of potassium is the favorable condition of Orange to grow
4. Papaya requires more than 30 degree to grow well whereas others required <= 30 degree
5. chickpea and kidneybeans humidity requires very less humidity to grow
6. All crops require more than pH value of 5 to grow
7. Rice requires very heavy rainfall (more than 200mm) where the muskmelon requires the least


In [ ]:
x = df.drop(['label'], axis=1)
x.head()

In [ ]:
y = df['label']
y.head()

In [ ]:
print('shape of x:', x.shape)
print('shape of y:', y.shape)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=0)


In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn.fit(x_train,y_train)
y_pred1 = clf_knn.predict(x_test)
print("Accuracy Score of KNN:",accuracy_score(y_test,y_pred1))

from sklearn.svm import SVC
clf_svc = SVC()
clf_svc.fit(x_train,y_train)
y_pred2 = clf_svc.predict(x_test)
print("Accuracy Score of SVC:",accuracy_score(y_test,y_pred2))

from sklearn.tree import DecisionTreeClassifier
clf_dtc = DecisionTreeClassifier(criterion='entropy',random_state=7)
clf_dtc.fit(x_train,y_train)
y_pred3 = clf_dtc.predict(x_test)
print("Accuracy Score of decision tree:",accuracy_score(y_test,y_pred3))

from sklearn.ensemble import RandomForestClassifier
clf_rfc = RandomForestClassifier(random_state=1)
clf_rfc.fit(x_train, y_train)
y_pred4 = clf_rfc.predict(x_test)
print("Accuracy Score of Random Forest:",accuracy_score(y_test,y_pred4))

Random forest gave us the best results

In [ ]:
y_train_pred = clf_rfc.predict(x_train)
print("Accuracy Score of Random Forest:",accuracy_score(y_train,y_train_pred))

In [ ]:
output = pd.DataFrame({'Real_class': y_test, 'Predicted_class': y_pred4})
output.head()

In [ ]:
input = np.array([[90,42,43,20.879744,82.002744,6.502985,202.935536]])
clf_rfc.predict(input)

In [ ]:
output.to_csv('Optimizing Agricultural Production.csv', index=False)
print("Submission was successfully saved!")